In [ ]:
# Author: Claudia Falsetti <1431314>
# Python 3.7.3 

import pandas as pd 
from nltk.corpus import stopwords, words, wordnet 
from gensim import corpora, models, similarities
from string import punctuation, digits
from collections import Counter, defaultdict
import re 
import numpy as np 
import csv 
import string 
from nltk.stem import WordNetLemmatizer 
import nltk 

from gensim import corpora, models, similarities
import pyLDAvis.gensim
import numpy as np 
from time import time 
import pickle 

In [ ]:
def get_documents(filename):

    data_frame = pd.read_csv(filename, names=["TweetID","Username", "Text"])
    df = data_frame.sort_values(by='Username').drop_duplicates('Text')
    data_frame_new = df.iloc[:,1:3]
    #print(data_frame_new.head(5))

    usernames = data_frame_new.iloc[:,0]

    usernames_list = set(usernames.values.tolist())


    documents = data_frame_new.groupby('Username')['Text'].apply(list).to_dict()

    return documents



In [ ]:
def preprocessing_data(documents):
    """
    Function that lowercase the text and clean it
    Break the sentences into tokens
    remove punctuations and stopwords
    Standardise the text : can't -> cannot, I'm -> I am
    """

    #tokenizer = RegexpTokenizer(r'\w+')

    token_frequency = defaultdict(int)
    lemmatizer = WordNetLemmatizer()
    lemmas = set(wordnet.all_lemma_names())

    stop_words = stopwords.words('english')

    stoplist_extra=['amp','youd', 'wed' ,'id', 'get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
            'pdt','pln','pst','wha','yep','yer','aest','didn','nzdt','via', 'seven', 'eight', 'nine', 'ten',
            'one','com','new','like','great','make','top','good','wow','yes','say','yay','would','thanks','thank','going',
            'new','use','should','could','best','really','see','want','nice', ' shes', 'hes ', 'were', 'theyre', 'yous',
            'two', 'three','four', 'five', 'six', 'while','know', 'ngl', 'brb', 'acc', 'smh', 'fwiw', 'ftl', 
            'lmao', 'lol', 'omg', 'https', 'mvp', 'isnt', 'arent', 'ever','cant', 'hnd', 'sbe', 'gsa', 'bwr', 'yeah', 'fuck', 'torture',
            'shit','say', 'try', 'well', 'take', 'way', 'many', 'yet','never', 'may', 'come', 'actually', 'much', 'thing', 'year', 'month',
            'people', 'also', 'around', 'think', 'keep', 'time', 'someone', 'give', 'back', 'need', 'time', 'feel', 'look', 'even', 
            'start', 'every', 'tell', 'first', 'lot', 'sure', 'though', 'end', 'still', 'bitch',
                   'wait', 'watch', 'already', 'always', 'thought','right', 'call', 'put', 'long',
                   'mak', 'cunt', 'aku', 'dia', 'yang','dah', 'masa', 'lepas', 'dari', 
                   'joseph', 'miller', 'ada', 'orang', 'kat', 'lah','zaman','gila','akan','tau','salah','mak','kali','ali',
                   'susah','lama','bahasa', 'het']

        # identify bigrams and unigrams to strip from tweets 

    counter = 0 

    with open('new_one.csv', 'a') as csvFile:
        csvWriter = csv.writer(csvFile)

        docs = []

        for username in documents.keys():

            #print("this is a key", username)
            counter += 1


           # print('Number of iterations: ' + str(counter) + "  out of:  " + str(len(usernames_list)))

            documents[username] = " ".join(documents[username]).lower().split()
            #documents[username] = [re.sub(r"em", "email ", str(documents[username]))]
        
            #documents[username] = [re.sub(r"til", "learned ", str(documents[username]))]
            documents[username] = [re.sub(r"fb", "facebook ", word) for word in documents[username]]   # look for twitter most used acronyms						
            documents[username] = [re.sub(r"jk", "joking ", word) for word in documents[username]]
            documents[username] = [re.sub(r'@[^\s]+','', word) for word in documents[username]]
            #documents[username] = [re.sub(r'#[^\s]+','', word) for word in documents[username]]


            # remove punctuation and replace with space and remove links 
            documents[username] = [re.sub(r'^https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in documents[username]]
            documents[username] = [re.sub(r"[\"'-,.;@?!&$:'/]+\ *", " ", word) for word in documents[username]]


            # remove digits 

            documents[username] = [re.sub(r'\w*\d\w*', '', word) for word in documents[username]]

            print("creating unigrams and stopwords list.....")

            # remove stopwords 
            documents[username] =  ' '.join(documents[username]).split()

            unigrams = [ word for word in documents[username] if len(word)==1]
            bigrams  = [ word for word in documents[username] if len(word)==2]
            stoplist  = set(stop_words + stoplist_extra + unigrams + bigrams)

            documents[username] = [word for word in documents[username] if word not in stoplist]


            #print("Stopwords have been removed ")
            
            documents[username] = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in documents[username]]
            documents[username] = [word for word in documents[username] if word in lemmas]
            documents[username] = [word for word in documents[username] if word not in stoplist]


            docs = [documents[username] for username in documents.keys()]

            csvWriter.writerow([username, documents[username]])
    csvFile.close()



    return docs

In [ ]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [ ]:
bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]

In [ ]:
def get_corpus(df):
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
